# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

### 문제 정의
> 문자 메시지 스팸 분류 문제<br>
> 1. 문자 메시지 데이터 분석
> 2. 스팸 분류 모델 성능 평가
### 학습 데이터
> * train/validation : spam.csv
> * test : spam_test_text.csv
### 변수 소개
> * text : 문자 메시지
> * label : 스팸여부

### References
> * 한국어 처리
>> * [konlpy - 한국어 처리 라이브러리](https://konlpy.org/ko/latest/)
>> * [한국어 품사 태그 비교표](https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0)
>> * [한국어 품사 태깅 성능 비교](https://konlpy.org/ko/latest/morph/#comparison-between-pos-tagging-classes)
>> * [한국어 시스템 사전](https://konlpy.org/ko/latest/data/#corpora)

> * 자연어 처리
>> * [NLTK](https://www.nltk.org/book/)
>> * [gensim](https://radimrehurek.com/gensim/)
>> * [Google guide](https://developers.google.com/machine-learning/guides/text-classification/step-2)
>> * [WordCloud](https://amueller.github.io/word_cloud/)

----

# 데이터 탐색부터 먼저 시작해보겠습니다.

---

### 0. 라이브러리 설치 및 불러오기

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

In [4]:
## import sklearn
import pandas as pd

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
#import tensorflow as tf
fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
#print(tf.config.list_physical_devices('GPU'))
#print(tf.config.list_logical_devices('GPU'))

### 1. 데이터 준비
#### 1-1. 데이터 가져오기

In [5]:
# 데이터를 가져옵니다.
import pandas as pd

df = pd.read_csv('./spam.csv')

### 1-2. 데이터 확인하기

In [6]:
# head 함수를 이용해 데이터를 확인해봅니다. (띄어쓰기가 되어 있지 않음을 알 수 있어요.)
df.head()

In [7]:
# info() 함수를 이용해서 데이터의 정보를 확인합니다.
df.info()

In [8]:
# label 데이터 분포를 확인합니다.
df.label.value_counts(normalize=True).plot.bar()

In [ ]:
print(f'정상의 비율 = {round(df.label.value_counts()[1]/len(df) * 100,3)}%')
print(f'스팸의 비율 = {round(df.label.value_counts()[0]/len(df) * 100,3)}%')

In [ ]:
# descibe 함수를 이용해서 기본 정보를 확인합니다.
df.describe()

#### 1-3. 결측치 제거

In [2]:
# 결측치를 제거한 후 확인합니다.
df = data.dropna(axis=0)
df.isnull().sum()

### 2. 텍스트 데이터 분석

#### 2-1. 텍스트 길이 분포 

In [17]:
# 텍스트 길이 분포와 최대 길이를 확인합니다.
import nltk

nltk_spam = nltk.Text(' '.join(df.text.explode()))
nltk_spam.plot(10)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_num_words_per_sample(sample_texts):
    num_words = [len(s.split()) for s in sample_texts]
    return np.median(num_words)

def plot_sample_length_distribution(sample_texts):
    plt.hist([len(s) for s in sample_texts], 50)
    plt.xlabel('Length of a sample')
    plt.ylabel('Number of samples')
    plt.title('Sample length distribution')
    plt.show()

print("NUMBER OF WORDS PER SAMPLE : ",get_num_words_per_sample(df.text))

plot_sample_length_distribution(df.text)

In [10]:
# konlpy를 활용하여 태깅 클래스를 불러옵니다.
from konlpy.tag import Mecab

mecab = Mecab()

#### 2-2. 형태소/명사 추출

In [11]:
# 태깅 클래스를 활용하여 형태소/명사를 추출합니다.
morphs = df.text.apply(lambda t: mecab.morphs(str(t)))
nouns = df.text.apply(lambda t: mecab.nouns(str(t)))

In [9]:
# Number of Samples / Number of words per sample 확인합니다.
# https://developers.google.com/machine-learning/guides/text-classification/step-2-5 참고
len(morphs)/morphs.apply(len).mean()

#### 2-3. NLTK Text로 변환

In [10]:
# Tokenize한 문자 데이터를 하나의 nltk.Text로 변환합니다.
spam_morphs_text = nltk.Text(morphs.explode().to_list())
spam_nouns_text = nltk.Text(nouns.explode().to_list())

#### 2-4. Frequency plot

In [11]:
# 형태소/명사 추출 각각 단어 분포를 확인합니다.
plt.figure(figsize=(8,4))
plt.title("형태소 개수 : " + str(len(spam_morphs_text.tokens)))
spam_morphs_text.plot(10)
plt.figure(figsize=(8,4))
plt.title("명사 개수 : " + str(len(spam_nouns_text.tokens)))
spam_nouns_text.plot(10)
plt.show()

#### 2-5. Similar words

In [ ]:
# 어떤 단어(스팸, 주식...)의 유사 단어를 확인합니다.
print(spam_morphs_text.similar('스팸'))
print(spam_nouns_text.similar('스팸'))

#### 2-6. Vocabulary

In [12]:
# 단어 사전을 확인합니다.
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
morphs_ft = cv.fit_transform(morphs)

cv.vocabulary_ 

#### 2-7. Collocation

In [13]:
# 연어(collocation)들을 확인합니다.
spam_morphs_text.collocations(window_size=5)
print("*"*50)
spam_nouns_text.collocations(window_size=5)

#### 3. WordCloud

#### 3-1. Morphs vs Nouns

In [15]:
# 형태소와 명사 추출 각각에 대해 wordcloud로 확인합니다.
import matplotlib.font_manager as font_manager
from wordcloud import WordCloud
NGC_path = font_manager.findfont('NanumGothicCoding')
#wc = WordCloud(font_path=NGC_path,max_font_size=40).generate(constitution)
wc = WordCloud(font_path=NGC_path,max_font_size=40,background_color="white", collocations=False).generate(' '.join(spam_morphs_text))
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
# 형태소와 명사 추출 각각에 대해 wordcloud로 확인합니다.
import matplotlib.font_manager as font_manager
from wordcloud import WordCloud
NGC_path = font_manager.findfont('NanumGothicCoding')
#wc = WordCloud(font_path=NGC_path,max_font_size=40).generate(constitution)
wc = WordCloud(font_path=NGC_path,max_font_size=40,background_color="white", collocations=False).generate(' '.join(spam_nouns_text))
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

#### 3-2. Ham vs Spam

In [16]:
# 명사 추출한 결과를 Ham과 Spam에 대해 wordcloud로 확인합니다.
spam = ' '.join(data.loc[data.label == 'spam', 'text'])
ham = ' '.join(data.loc[data.label == 'ham', 'text'])
nouns_spam = mecab.nouns(spam)
nouns_ham = mecab.nouns(ham)
nltk_nouns_spam = nltk.Text(nouns_spam)
nltk_nouns_ham = nltk.Text(nouns_ham)

In [ ]:
# 명사 추출한 결과를 Ham과 Spam에 대해 wordcloud로 확인합니다.
# spam
import matplotlib.font_manager as font_manager
from wordcloud import WordCloud
NGC_path = font_manager.findfont('NanumGothicCoding')
#wc = WordCloud(font_path=NGC_path,max_font_size=40).generate(constitution)
wc = WordCloud(font_path=NGC_path,max_font_size=40,background_color="white", collocations=False).generate(' '.join(nltk_nouns_spam))
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
# 명사 추출한 결과를 Ham과 Spam에 대해 wordcloud로 확인합니다.
# ham
import matplotlib.font_manager as font_manager
from wordcloud import WordCloud
NGC_path = font_manager.findfont('NanumGothicCoding')
#wc = WordCloud(font_path=NGC_path,max_font_size=40).generate(constitution)
wc = WordCloud(font_path=NGC_path,max_font_size=40,background_color="white", collocations=False).generate(' '.join(nltk_nouns_ham))
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
# 정상 문자에서 신고, 스팸이란 단어가 많이 나옴